In [86]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [87]:
data = pd.read_csv("./data/patient_timepoints.csv")
data.head(10)

/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_16879/618099685.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./data/patient_timepoints.csv")


,patient_id,timepoint,protein_0,protein_1,protein_2,protein_3,protein_4,protein_5,protein_6,protein_7,...,protein_40911,protein_40912,protein_40913,protein_40914,protein_40915,protein_40916,protein_40917,protein_40918,protein_40919,protein_40920
0,1,1,2.173878e+09,2.324375e+09,7.971150e+09,6.637975e+09,2.310100e+09,2.938325e+09,6.205375e+09,1.433934e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,2.197450e+09,2.086445e+09,5.333875e+09,3.353400e+09,2.704025e+09,1.658998e+09,3.136500e+09,3.421775e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,2.864495e+09,1.523392e+09,5.175725e+09,3.819700e+09,1.235965e+09,1.392411e+09,2.677950e+09,2.232474e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,4,1.613750e+09,9.303475e+08,4.358510e+09,3.478500e+09,1.761028e+09,1.200268e+09,5.770850e+09,1.383948e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,5,2.574530e+09,1.654390e+09,7.262825e+09,4.518775e+09,9.585625e+08,4.677333e+09,9.808875e+09,1.871452e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,6,4.678075e+09,2.138075e+09,9.100750e+09,6.012960e+09,1.503225e+09,7.990575e+09,9.776675e+09,2.107112e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,7,3.746350e+09,2.543175e+09,7.505575e+09,5.263800e+09,5.039025e+09,6.101225e+09,1.203450e+10,3.658275e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2,1,1.429182e+09,1.363535e+09,5.677100e+09,3.571750e+09,1.866490e+09,1.009922e+09,9.080450e+09,1.289592e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2,2,3.215465e+09,2.189188e+09,4.854168e+09,4.289225e+09,1.764602e+09,2.154432e+09,3.744925e+09,1.237022e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2,3,1.971002e+09,1.047142e+09,3.649325e+09,2.088325e+09,1.309340e+09,4.863910e+08,2.659812e+09,1.320202e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
data.shape

(336, 40923)

In [90]:
data['timepoint'].unique()

array([1, 2, 3, 4, 5, 6, 7])

In [91]:
data['patient_id'].unique()

array([1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, '57', '58',
       'F1_healthyFemale', 'F2_healthyFemale', 'F3_healthyFemale',
       'M1_healthyMale', 'M2_healthyMale', 'M3_healthyMale'], dtype=object)

In [92]:
print(data.dtypes)

patient_id        object
timepoint          int64
protein_0        float64
protein_1        float64
protein_2        float64
                  ...   
protein_40916    float64
protein_40917    float64
protein_40918    float64
protein_40919    float64
protein_40920    float64
Length: 40923, dtype: object


In [93]:
mapping = {
    'F1_healthyFemale': 59,
    'F2_healthyFemale': 60,
    'F3_healthyFemale': 61,
    'M1_healthyMale': 62,
    'M2_healthyMale': 63,
    'M3_healthyMale': 64
}

data['patient_id'].replace(mapping, inplace=True)

print(data['patient_id'].unique())


[1 2 3 5 6 7 8 9 10 11 12 14 15 16 17 19 20 21 22 23 24 25 26 27 28 29 30
 31 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 '57'
 '58' 59 60 61 62 63 64]


/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_16879/3951797265.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['patient_id'].replace(mapping, inplace=True)


In [95]:
data['patient_id'] = pd.to_numeric(data['patient_id'], errors='coerce')
data['patient_id'] = data['patient_id'].astype(int)
print(data['patient_id'].unique())


[ 1  2  3  5  6  7  8  9 10 11 12 14 15 16 17 19 20 21 22 23 24 25 26 27
 28 29 30 31 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
 53 54 57 58 59 60 61 62 63 64]


In [104]:
data[data['patient_id'] == 63]

,patient_id,timepoint,protein_0,protein_1,protein_2,protein_3,protein_4,protein_5,protein_6,protein_7,...,protein_40911,protein_40912,protein_40913,protein_40914,protein_40915,protein_40916,protein_40917,protein_40918,protein_40919,protein_40920
330,63,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
331,63,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332,63,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

patient_ids = range(1, 65)
desired_timestamps = [1, 2, 3, 4, 5, 6, 7]

missing_rows = []

for patient_id in patient_ids:
    existing_timestamps = data[data['patient_id'] == patient_id]['timepoint'].values

    for timestamp in desired_timestamps:
        if timestamp not in existing_timestamps:
            new_row = {col: 0 for col in data.columns if col != 'patient_id' and col != 'timepoint'}  # Set all columns to 0
            new_row['patient_id'] = patient_id
            new_row['timepoint'] = timestamp
            missing_rows.append(new_row)

if missing_rows:
    missing_df = pd.DataFrame(missing_rows)
    data = pd.concat([data, missing_df], ignore_index=True)

# just creating all timestamps for each patient, as some didn't have it

In [110]:
df_pivoted = data.set_index(['patient_id', 'timepoint'])
df_pivoted = df_pivoted.unstack(level='timepoint')
df_pivoted.head(10)

protein_0                                            \
timepoint              1             2             3             4   
patient_id                                                           
1           2.173878e+09  2.197450e+09  2.864495e+09  1.613750e+09   
2           1.429182e+09  3.215465e+09  1.971002e+09  2.619665e+09   
3           4.075575e+09  4.111725e+09  3.200225e+09  2.730582e+09   
4           0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
5           3.568100e+09  3.992115e+09  3.082350e+09  3.500050e+09   
6           3.477745e+09  3.020890e+09  4.193715e+09  5.275950e+09   
7           2.938825e+09  3.493050e+09  2.090175e+09  2.510425e+09   
8           2.181075e+09  2.019118e+09  2.547425e+09  2.198400e+09   
9           3.046400e+09  2.785075e+09  3.139700e+09  3.101350e+09   
10          3.367258e+09  3.600752e+09  4.258425e+09  2.306745e+09   

                                                         protein_1  \
timepoint              5             6             7             1   
patient_id                                                           
1           2.574530e+09  4.678075e+09  3.746350e+09  2.324375e+09   
2           4.389300e+09  4.080025e+09  4.566405e+09  1.363535e+09   
3           3.550925e+09  4.502175e+09  2.639325e+09  1.664725e+09   
4           0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
5           4.845520e+09  2.310480e+09  3.684542e+09  2.511300e+09   
6           4.490525e+09  3.970550e+09  4.514300e+09  1.791998e+09   
7           5.215050e+09  4.757975e+09  4.009175e+09  2.765140e+09   
8           3.557575e+09  2.599375e+09  2.668900e+09  1.267408e+09   
9           4.616875e+09  3.346750e+09  4.202175e+09  2.649550e+09   
10          2.768568e+09  3.394802e+09  3.852768e+09  1.770512e+09   

                                        ... protein_40919            \
timepoint              2             3  ...             5    6    7   
patient_id                              ...                           
1           2.086445e+09  1.523392e+09  ...           0.0  0.0  0.0   
2           2.189188e+09  1.047142e+09  ...           0.0  0.0  0.0   
3           1.775125e+09  1.616212e+09  ...           0.0  0.0  0.0   
4           0.000000e+00  0.000000e+00  ...           0.0  0.0  0.0   
5           1.907455e+09  1.481990e+09  ...           0.0  0.0  0.0   
6           1.195212e+09  1.661865e+09  ...           0.0  0.0  0.0   
7           1.704800e+09  2.185925e+09  ...           0.0  0.0  0.0   
8           8.989070e+08  1.378238e+09  ...           0.0  0.0  0.0   
9           2.596525e+09  2.895650e+09  ...           0.0  0.0  0.0   
10          1.118813e+09  2.593525e+09  ...           0.0  0.0  0.0   

           protein_40920                                
timepoint              1    2    3    4    5    6    7  
patient_id                                              
1                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
6                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
7                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
8                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
9                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  
10                   0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[10 rows x 286447 columns]

In [111]:
df_pivoted = df_pivoted.fillna(0)  # Or use NaN if you prefer


In [112]:
df_pivoted.shape

(64, 286447)

In [113]:
df_pivoted.columns = ['{}_tp{}'.format(col, tp) for col, tp in df_pivoted.columns]
df_pivoted = df_pivoted.reset_index()
df_pivoted.head(10)

,patient_id,protein_0_tp1,protein_0_tp2,protein_0_tp3,protein_0_tp4,protein_0_tp5,protein_0_tp6,protein_0_tp7,protein_1_tp1,protein_1_tp2,...,protein_40919_tp5,protein_40919_tp6,protein_40919_tp7,protein_40920_tp1,protein_40920_tp2,protein_40920_tp3,protein_40920_tp4,protein_40920_tp5,protein_40920_tp6,protein_40920_tp7
0,1,2.173878e+09,2.197450e+09,2.864495e+09,1.613750e+09,2.574530e+09,4.678075e+09,3.746350e+09,2.324375e+09,2.086445e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.429182e+09,3.215465e+09,1.971002e+09,2.619665e+09,4.389300e+09,4.080025e+09,4.566405e+09,1.363535e+09,2.189188e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,4.075575e+09,4.111725e+09,3.200225e+09,2.730582e+09,3.550925e+09,4.502175e+09,2.639325e+09,1.664725e+09,1.775125e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,3.568100e+09,3.992115e+09,3.082350e+09,3.500050e+09,4.845520e+09,2.310480e+09,3.684542e+09,2.511300e+09,1.907455e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,3.477745e+09,3.020890e+09,4.193715e+09,5.275950e+09,4.490525e+09,3.970550e+09,4.514300e+09,1.791998e+09,1.195212e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,2.938825e+09,3.493050e+09,2.090175e+09,2.510425e+09,5.215050e+09,4.757975e+09,4.009175e+09,2.765140e+09,1.704800e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,2.181075e+09,2.019118e+09,2.547425e+09,2.198400e+09,3.557575e+09,2.599375e+09,2.668900e+09,1.267408e+09,8.989070e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,3.046400e+09,2.785075e+09,3.139700e+09,3.101350e+09,4.616875e+09,3.346750e+09,4.202175e+09,2.649550e+09,2.596525e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,3.367258e+09,3.600752e+09,4.258425e+09,2.306745e+09,2.768568e+09,3.394802e+09,3.852768e+09,1.770512e+09,1.118813e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
df_pivoted.shape

(64, 286448)

In [115]:
df_pivoted = df_pivoted.fillna(0)


In [ ]:

class DeepAutoencoder(nn.Module):
    def __init__(self, input_dim=286448, embedding_dim=128):
        super(DeepAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, embedding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(embedding_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.Linear(4096, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

    def get_embedding(self, x):
        with torch.no_grad():
            return self.encoder(x)


In [78]:
df_pivoted = df_pivoted.fillna(0)
patient_ids = df_pivoted['patient_id'].values

X = df_pivoted.drop(columns=['patient_id'])

X_tensor = torch.tensor(X.values, dtype=torch.float32)
X_tensor.shape

torch.Size([58, 286447])

In [34]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

dataset = TensorDataset(X_tensor)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

model = DeepAutoencoder(input_dim=X_tensor.shape[1], embedding_dim=128)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
epochs = 100
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in loader:
        batch_x = batch[0]
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_x)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")


KeyboardInterrupt: 